#### 1. D Sector 구매 이력 고객 중, C Sector 미구매 고객 Cross-Selling
#### 2. DAB 구매 후 3개월 이상 미구매 고객(휴면 고객) 대상 CBA Cross-Selling
#### 3.  마지막 구매 DAA, 월령 12개월 이상 구매 고객 DAB 추천 Cross-Selling
#### 4. DAB구매 고객 대상 DAD Up-Selling

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('train_data_baby.xlsx')
df

,Customer_ID,Order_Date,Product_No,Price,Sector,CTG,Sub_CTG,Month
0,956846,2020-04-06,16421,15450,C,CA,CBA,27
1,938614,2020-04-06,13350,8050,C,CB,CBA,19
2,333195,2020-04-06,14591,15100,C,CC,CCB,19
3,338999,2020-04-06,5757,15850,D,DA,DAB,7
4,463956,2020-04-06,11866,28585,D,DA,DAB,36
...,...,...,...,...,...,...,...,...
408153,920701,2020-10-01,15872,10380,C,CB,CBA,19
408154,908654,2020-10-01,10803,14305,C,CC,CCB,17
408155,649154,2020-10-01,19326,7450,C,CA,CBA,23
408156,954420,2020-10-01,18910,20925,D,DA,DAB,16


In [3]:
df['CTG'].unique()

array(['CA', 'CB', 'CC', 'DA'], dtype=object)

#### 1. D Sector 구매 이력 고객 중, C Sector 미구매 고객 Cross-Selling

In [4]:
# 각 CTG별 구매 횟수 확인
df_count = df.pivot_table(index = 'Customer_ID', columns = 'CTG', values = 'Price', aggfunc = 'count')
df_count2 = df_count.fillna(0).astype(int)

In [5]:
df_count2

CTG,CA,CB,CC,DA
Customer_ID,,,,
100018,0,0,0,1
100021,0,0,0,3
100042,0,0,0,1
100051,0,0,0,1
100080,0,1,0,5
...,...,...,...,...
999935,0,0,0,2
999961,0,2,4,14
999987,0,0,0,2


In [15]:
# 자녀 월령 정보 Merge
df_count3 = pd.merge(df[['Customer_ID','Month']] ,df_count2, right_index = True, left_on = 'Customer_ID', how = 'left')
df_count3

,Customer_ID,Month,CA,CB,CC,DA
0,956846,27,1,0,0,0
1,938614,19,0,1,1,0
2,333195,19,0,0,1,0
3,338999,7,0,0,0,1
4,463956,36,0,0,1,11
...,...,...,...,...,...,...
408153,920701,19,0,13,0,2
408154,908654,17,0,0,3,2
408155,649154,23,1,0,0,1
408156,954420,16,16,3,2,24


In [16]:
# D 구매 이력 고객 중 C 구매 0인 고객 Filtering
df_count3_filtered = df_count3[(df_count3['DA'] > 0) & (df_count3['CA'] == 0) & (df_count3['CB'] == 0) & (df_count3['CC'] == 0)]

In [17]:
# 자녀 월령에 적합한 C Sector내 CTG 추천
def Recommend1(x):
    if x <= 6:
        return 'CA'
    elif x <= 18:
        return 'CB'
    else:
        return 'CC'

df_count3_filtered['Recommend1'] = df_count3_filtered['Month'].apply(Recommend1)

C:\Users\J13868\AppData\Local\Temp\ipykernel_26772\1971321592.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_count3_filtered['Recommend1'] = df_count3_filtered['Month'].apply(Recommend1)


In [19]:
df_count3_filtered2 = df_count3_filtered.set_index('Customer_ID')

In [20]:
df_count3_filtered2

,Month,CA,CB,CC,DA,Recommend1
Customer_ID,,,,,,
338999,7,0,0,0,1,CB
546233,10,0,0,0,8,CB
331960,8,0,0,0,7,CB
331960,8,0,0,0,7,CB
414364,20,0,0,0,1,CC
...,...,...,...,...,...,...
830934,11,0,0,0,16,CB
830934,11,0,0,0,16,CB
550401,20,0,0,0,2,CC


#### 2. DAB 구매 후 3개월 이상 미구매 고객(휴면 고객) 대상 CBA 추천

In [21]:
# 미구매 3개월 이상 경과 고객 filtering
df_last = df.pivot_table(index = ['Customer_ID','Sub_CTG'], values = 'Order_Date', aggfunc = 'max')
df_last2 = df_last.reset_index(level = 'Sub_CTG')
df_last3 = df_last2[(df_last2['Order_Date'] < '2020-07-01') & (df_last2['Sub_CTG'] == 'DAB')]

In [22]:
df_last4 = df_last3.rename(columns = {'Sub_CTG' : 'Recommend2'})
df_last4

,Recommend2,Order_Date
Customer_ID,,
100018,DAB,2020-05-16
100042,DAB,2020-06-30
100137,DAB,2020-05-02
100238,DAB,2020-05-09
100270,DAB,2020-06-03
...,...,...
999739,DAB,2020-05-16
999853,DAB,2020-05-11
999866,DAB,2020-05-02


#### 3.  마지막 구매 DAA, 월령 12개월 이상 구매 고객 DAB 추천

In [24]:
df_last_month = df.pivot_table(index = ['Customer_ID','Month', 'Sub_CTG'], values = 'Order_Date', aggfunc = 'max')

In [25]:
df_last_month2 = df_last_month.reset_index(level = ['Month', 'Sub_CTG'])

In [26]:
df_last_month3 = df_last_month2[(df_last_month2['Sub_CTG'] == 'DAA') & (df_last_month2['Month'] >= 12)]

In [27]:
df_last_month3['Recommend3'] = 'DAB'
df_last_month3

C:\Users\J13868\AppData\Local\Temp\ipykernel_26772\4091079257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last_month3['Recommend3'] = 'DAB'


,Month,Sub_CTG,Order_Date,Recommend3
Customer_ID,,,,
100021,35,DAA,2020-04-14,DAB
100051,26,DAA,2020-04-22,DAB
100155,26,DAA,2020-06-25,DAB
100189,33,DAA,2020-07-07,DAB
100191,27,DAA,2020-09-11,DAB
...,...,...,...,...
999825,22,DAA,2020-05-20,DAB
999873,13,DAA,2020-05-07,DAB
999961,16,DAA,2020-08-28,DAB


#### 4. DAB구매 고객 대상 DAD Up-Selling

In [28]:
# DAB 구매 고객 중 DAD 미구매 고객 Filtering
df_pivot = df.pivot_table(index = 'Customer_ID', columns = 'Sub_CTG', values = 'Price', aggfunc = 'count')

In [29]:
df_pivot2 = df_pivot.fillna(0).astype(int)

In [30]:
df_pivot3 = df_pivot2[(df_pivot2['DAB'] > 0) & (df_pivot2['DAD'] == 0)]

In [31]:
df_pivot3['Recommend4'] = 'DAD'

C:\Users\J13868\AppData\Local\Temp\ipykernel_26772\757724241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pivot3['Recommend4'] = 'DAD'


In [32]:
df_pivot3[['Recommend4']]

Sub_CTG,Recommend4
Customer_ID,
100018,DAD
100042,DAD
100080,DAD
100086,DAD
100137,DAD
...,...
999925,DAD
999935,DAD
999961,DAD


In [45]:
df_result1 = pd.merge(df_count3[['Customer_ID','Month']].set_index('Customer_ID').sort_values(by = 'Customer_ID'), df_count3_filtered2['Recommend1'], on = 'Customer_ID' , how = 'left')
df_result2 = pd.merge(df_result1,df_last4[['Recommend2']],  on = 'Customer_ID' , how = 'left')
df_result3 = pd.merge(df_result2,df_last_month3[['Recommend3']],  on = 'Customer_ID' , how = 'left')
df_result4 = pd.merge(df_result3,df_pivot3[['Recommend4']],  on = 'Customer_ID' , how = 'left')

In [46]:
df_result5 = df_result4.fillna('')

In [47]:
df_result5['Recommendation'] = df_result5['Recommend1'] + ' ' + df_result5['Recommend2'] + ' ' + df_result5['Recommend3'] + ' ' + df_result5['Recommend4']

In [48]:
df_result = df_result5[['Month', 'Recommendation']]
df_result

,Month,Recommendation
Customer_ID,,
100018,32,CC DAB DAD
100021,35,CC DAB
100021,35,CC DAB
100021,35,CC DAB
100021,35,CC DAB
...,...,...
999994,12,DAB DAD
999998,25,CC DAD
999998,25,CC DAD
